In [ ]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA
import pandas as pd
import sys
# 找不到module
sys.path.append('/Users/max/mypython/python-demo')
print(sys.path)

# 创建策略
class SmaCross(Strategy):
    def init(self):
        price = self.data.Close
        self.ma1 = self.I(SMA, price, 10)
        self.ma2 = self.I(SMA, price, 20)

    def next(self):
        if crossover(self.ma1, self.ma2):
            self.buy()
        elif crossover(self.ma2, self.ma1):
            self.sell()

In [ ]:
# 跑数据
data = pd.read_csv("/Users/max/.tibet/cn/features/SZ300760/day.csv", index_col='datetime', parse_dates=True, infer_datetime_format=True) 
data = data.loc['2022-06-01':'2022-09-30']
# Backtest columns
data.columns = ['Open', 'High', 'Low', 'Close', 'Volume']


In [19]:
from data import eastmoney as et
json = et.chartbar_json('SZ300760','2020-01-01', '2022-10-25', 1)
df = et.to_df(json)
data = df.iloc[:, :5] # 取钱5列
data.columns = ['Open', 'High', 'Low', 'Close', 'Volume']


In [37]:
from datetime import datetime
df = data.loc['2021-01-01':]
# df = data.dropna()
df = df.apply(pd.to_numeric)
df.index = df.index.map(lambda x: datetime.strptime(x,'%Y-%m-%d'))
# isinstance(df.index[0],datetime)

# columns = df.columns
# df[columns] = df[columns].apply(pd.to_numeric,errors='coerce')
# df

In [ ]:
bt = Backtest(df, SmaCross, commission=.002, exclusive_orders=True)
stats = bt.run()
bt.plot()